In [ ]:
import sys, os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/MyDrive/faceData.csv')

Mounted at /content/gdrive


In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/faceData.csv')

In [ ]:
X_train,train_y,X_test,test_y=[],[],[],[]
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [ ]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')
train_y=np_utils.to_categorical(train_y, num_classes=7)
test_y=np_utils.to_categorical(test_y, num_classes=7)

In [ ]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [ ]:
cnn = Sequential()

cnn.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(48,48,1)))
cnn.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn.add(Dropout(0.25))


cnn.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
cnn.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn.add(Dropout(0.25))


cnn.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
cnn.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn.add(Dropout(0.5))


cnn.add(Flatten())

cnn.add(Dense(512, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(512, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(512, activation='relu'))
cnn.add(Dropout(0.5))

cnn.add(Dense(7, activation='softmax'))

cnn.summary()

cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 24, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 256)       5

In [ ]:
#Compliling the model
cnn.compile(loss=categorical_crossentropy,optimizer=Adam(learning_rate=0.00001),metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=300, verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
cnn.fit(X_train, train_y,batch_size=120,epochs=300,validation_data=(X_test, test_y),callbacks=[monitor],shuffle=True)

Epoch 1/300
240/240 [==============================] - 90s 166ms/step - loss: 2.8383 - accuracy: 0.1567 - val_loss: 1.8495 - val_accuracy: 0.2204
Epoch 2/300
240/240 [==============================] - 33s 139ms/step - loss: 2.2984 - accuracy: 0.1825 - val_loss: 1.8391 - val_accuracy: 0.2530
Epoch 3/300
240/240 [==============================] - 34s 141ms/step - loss: 2.1165 - accuracy: 0.1942 - val_loss: 1.8389 - val_accuracy: 0.2499
Epoch 4/300
240/240 [==============================] - 34s 143ms/step - loss: 2.0008 - accuracy: 0.2030 - val_loss: 1.8394 - val_accuracy: 0.2597
Epoch 5/300
240/240 [==============================] - 34s 141ms/step - loss: 1.9584 - accuracy: 0.2025 - val_loss: 1.8385 - val_accuracy: 0.2594
Epoch 6/300
240/240 [==============================] - 34s 142ms/step - loss: 1.9164 - accuracy: 0.2123 - val_loss: 1.8362 - val_accuracy: 0.2563
Epoch 7/300
240/240 [==============================] - 34s 142ms/step - loss: 1.8947 - accuracy: 0.2140 - val_loss: 1.8318 -

In [ ]:
FacialModelArchitecture_json = cnn.to_json()
with open("ModelArchitecture.json", "w") as json_file:
    json_file.write(FacialModelArchitecture_json)
cnn.save_weights("FacialModelWeights.h5")